### Ejercicio 22: Fertilizantes

Una empresa de fertilizantes dispone de ciertas cantidades de nitrato, fosfato y potasa indicadas en la tabla adjunta que utiliza para fabricar tres tipos de fertilizantes A, B y C mezclandolos
con un producto base del que dispone de cantidades en exceso. Las composiciones de cada uno de
los fertizantes figuran tambien en la tabla adjunta, así como los precios de venta de los fertizantes
A, B y C. El costo de realizar la mezcla, embalaje y venta son iguales para los tres y ascienden a 6
=C/Tm. Si la empresa por determinadas razones no puede producir mas de 6000 Tm del producto
A y sabe que el mercado no está saturado, se desea calcular cuanto fertilizante de cada tipo debe
fabricar y cuanta materia prima utilizar para obtener el máximo beneficio en un cierto periodo
de tiempo. ¿Cuál es el beneficio óptimo


#### Importamos Pyomo

In [103]:
from pyomo.environ import *
import pandas as pd
import numpy as np


In [104]:
df_comp = pd.DataFrame({'Nitrato': [0.05,0.05,0.1],'Fosfato': [0.1,0.1,0.1],'Potasa':[0.05,0.1,0.1],'Base':[0.8,0.75,0.70]}, index = ['A','B','C'])
df_comp 

,Nitrato,Fosfato,Potasa,Base
A,0.05,0.1,0.05,0.80
B,0.05,0.1,0.10,0.75
C,0.10,0.1,0.10,0.70


In [105]:
df_precio = pd.DataFrame({'Precio': [16,20,24]}, index = ['A','B','C'])
df_precio

,Precio
A,16
B,20
C,24


In [106]:
df_disp = pd.DataFrame({'Disp': [1000,1800,1200,99990]}, index = ['Nitrato','Fosfato','Potasa','Base'])
df_disp

,Disp
Nitrato,1000
Fosfato,1800
Potasa,1200
Base,99990


In [107]:
df_cost = pd.DataFrame({'Coste': [64,16,40,2]}, index = ['Nitrato','Fosfato','Potasa','Base'])
df_cost

,Coste
Nitrato,64
Fosfato,16
Potasa,40
Base,2


#### Creamos un problema concreto, es decir, un objeto de la clase problema que ya contenga parámetros

In [108]:
model = ConcreteModel()

#### Necesitamos utilizar sets

In [109]:
model.f=Set(initialize=df_comp.index)
model.c=Set(initialize=df_cost.index)

#### Definimos las variables

In [110]:
model.Fin=Var(model.c, model.f, within=NonNegativeReals)
model.Fc=Var(model.c,  within=NonNegativeReals)
model.Ff=Var(model.f,  within=NonNegativeReals)

#### Definimos la función objetivo

In [111]:
model.obj = Objective(expr = sum(model.Ff[f]*(df_precio['Precio'][f]-6) for f in model.f)-sum(model.Fc[c]*df_cost['Coste'][c] for c in model.c), sense=maximize)

#### Definimos la restricciones

In [112]:
model.componentes = ConstraintList()
for c in model.c:
    model.componentes.add(
        sum(model.Fin[c,f] for f in model.f)==model.Fc[c]
    )

model.disponib = ConstraintList()
for c in model.c:
    model.disponib.add(
        df_disp['Disp'][c]>=model.Fc[c]
    )

model.constA = Constraint(expr= model.Ff['A']<=6000)  

model.fert = ConstraintList()
for f in model.f:
    model.fert.add(
        sum(model.Fin[c,f] for c in model.c)==model.Ff[f]
    )

model.composicion = ConstraintList()
for c in model.c:
    model.composicion.add(
        sum(model.Ff[f]*df_comp[c][f] for f in model.f)==model.Fc[c]
    )

#### Resolvemos el problema

In [113]:
results = SolverFactory('glpk').solve(model)
model.pprint()
results.write()   

7 Set Declarations
    Fin_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    c*f :   12 : {('Nitrato', 'A'), ('Nitrato', 'B'), ('Nitrato', 'C'), ('Fosfato', 'A'), ('Fosfato', 'B'), ('Fosfato', 'C'), ('Potasa', 'A'), ('Potasa', 'B'), ('Potasa', 'C'), ('Base', 'A'), ('Base', 'B'), ('Base', 'C')}
    c : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {'Nitrato', 'Fosfato', 'Potasa', 'Base'}
    componentes_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {1, 2, 3, 4}
    composicion_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {1, 2, 3, 4}
    disponib_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {